# Test

Test con test set

In [8]:
# read files in paper_results directory 
import os
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from datasets import load_dataset, concatenate_datasets, DatasetDict


def objective_function(x, res, ground_truth):
    a, b, c = x
    return np.sum((a * res[0] + b * res[1] + c * res[2] - ground_truth) ** 2)

def constraint(x):
    return np.sum(x) - 1

results = []


for n in range(5):

    np.random.seed(42)
    use_mnli = True
    result_path = 'paper_results_mnli/' if use_mnli else 'paper_results/'
    path = '/data1/malto/shroom'


    baseline_test = pd.read_csv(result_path + '/baseline_test' + str(n) + '.csv')['baseline'].to_numpy()
    baseline_val = pd.read_csv(result_path + '/baseline_val' + str(n) + '.csv')['baseline'].to_numpy()
    crlft_test = pd.read_csv(result_path + '/crlft_test' + str(n) + '.csv')['crlft'].to_numpy()
    crlft_val = pd.read_csv(result_path + '/crlft_val' + str(n) + '.csv')['crlft'].to_numpy()
    sequential_test = pd.read_csv(result_path + '/sequential_test' + str(n) + '.csv')['sequential'].to_numpy() 
    sequential_val = pd.read_csv(result_path + '/sequential_val' + str(n) + '.csv')['sequential'].to_numpy()

    labels_val = pd.read_json(path + '/val.model-aware.json', lines=True)['p(Hallucination)'].to_numpy()
    # read test.model-agnostic.json 

    res = np.stack((baseline_test, sequential_test, crlft_test), axis=0)
    test = load_dataset("json", data_files=["test.model-agnostic.json"])
    # shuffle the dataset
    ground_truth = test['train']['p(Hallucination)']
    ground_truth = np.array(ground_truth)

    # shuffle in the same way the ground truth and the predictions
    indices = np.arange(len(ground_truth))
    np.random.shuffle(indices)

    ground_truth = ground_truth[indices]
    res = res[:, indices]

    gt_ensemble = ground_truth[:500]
    res_ensemble = res[:, :500]

    gt_res = ground_truth[500:]
    res_res = res[:, 500:]

    con = {'type': 'eq', 'fun': constraint}
    bounds = [(0, 1), (0, 1), (0, 1)]  # a, b, c >= 0 and <= 1
    x0 = np.array([1/3, 1/3, 1/3])

    result = minimize(objective_function, x0, args=(res, ground_truth), bounds=bounds, constraints=con)

    print("Optimal values:")
    print("a =", result.x[0])
    print("b =", result.x[1])
    print("c =", result.x[2])
    print("Objective function value:", result.fun)

    r1_new = result.x[0] * res_res[0]
    r2_new = result.x[1] * res_res[1]
    r3_new = result.x[2] * res_res[2]

    res = r1_new + r2_new + r3_new

    predictions = np.where(res > 0.5, 1, 0)
    labels = np.where(gt_res > 0.5, 1, 0)
    print(np.sum(predictions == labels)/len(labels))
    results.append(np.sum(predictions == labels)/len(labels))

Optimal values:
a = 0.16547166201172253
b = 0.2621734376092141
c = 0.5723549003790634
Objective function value: 101.4890207148955
0.813
Optimal values:
a = 0.14837527182057303
b = 0.45609284180985166
c = 0.395531886369845
Objective function value: 110.92592615328114
0.813
Optimal values:
a = 0.14622628904763496
b = 0.44601407869662346
c = 0.40775963225601897
Objective function value: 115.37088016486558
0.807
Optimal values:
a = 0.1167475764690003
b = 0.3726865118334598
c = 0.5105659116977526
Objective function value: 112.84955120448384
0.813
Optimal values:
a = 0.06388861216096206
b = 0.45613395594980916
c = 0.4799774318892288
Objective function value: 104.71661808345706
0.815


In [9]:
print(np.mean(results)*100, np.std(results)*100)

81.22 0.27129319932500673


In [5]:
# read files in paper_results directory 
import os
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from datasets import load_dataset, concatenate_datasets, DatasetDict


def objective_function(x, res, ground_truth):
    a, b, c = x
    return np.sum((a * res[0] + b * res[1] + c * res[2] - ground_truth) ** 2)

def constraint(x):
    return np.sum(x) - 1

results = []


for n in range(5):

    np.random.seed(42)
    use_mnli = False
    result_path = 'paper_results_mnli/' if use_mnli else 'paper_results/'
    path = '/data1/malto/shroom'


    baseline_test = pd.read_csv(result_path + '/baseline_test' + str(n) + '.csv')['baseline'].to_numpy()
    baseline_val = pd.read_csv(result_path + '/baseline_val' + str(n) + '.csv')['baseline'].to_numpy()
    crlft_test = pd.read_csv(result_path + '/crlft_test' + str(n) + '.csv')['crlft'].to_numpy()
    crlft_val = pd.read_csv(result_path + '/crlft_val' + str(n) + '.csv')['crlft'].to_numpy()
    sequential_test = pd.read_csv(result_path + '/sequential_test' + str(n) + '.csv')['sequential'].to_numpy() 
    sequential_val = pd.read_csv(result_path + '/sequential_val' + str(n) + '.csv')['sequential'].to_numpy()

    labels_val = pd.read_json(path + '/val.model-aware.json', lines=True)['p(Hallucination)'].to_numpy()
    # read test.model-agnostic.json 

    res = np.stack((baseline_test, sequential_test, crlft_test), axis=0)
    test = load_dataset("json", data_files=["test.model-agnostic.json"])
    # shuffle the dataset
    ground_truth = test['train']['p(Hallucination)']
    ground_truth = np.array(ground_truth)

    # shuffle in the same way the ground truth and the predictions
    indices = np.arange(len(ground_truth))
    np.random.shuffle(indices)

    ground_truth = ground_truth[indices]
    res = res[:, indices]

    gt_ensemble = ground_truth[:500]
    res_ensemble = res[:, :500]

    gt_res = ground_truth[500:]
    res_res = res[:, 500:]

    con = {'type': 'eq', 'fun': constraint}
    bounds = [(0, 1), (0, 1), (0, 1)]  # a, b, c >= 0 and <= 1
    x0 = np.array([1/3, 1/3, 1/3])

    result = minimize(objective_function, x0, args=(res, ground_truth), bounds=bounds, constraints=con)

    print("Optimal values:")
    print("a =", result.x[0])
    print("b =", result.x[1])
    print("c =", result.x[2])
    print("Objective function value:", result.fun)

    r1_new = result.x[0] * res_res[0]
    r2_new = result.x[1] * res_res[1]
    r3_new = result.x[2] * res_res[2]

    res = r1_new + r2_new + r3_new

    predictions = np.where(res > 0.5, 1, 0)
    labels = np.where(gt_res > 0.5, 1, 0)
    print(np.sum(predictions == labels)/len(labels))
    results.append(np.sum(predictions == labels)/len(labels))

Optimal values:
a = 0.21827181614178287
b = 0.5299948383258719
c = 0.2517333455323452
Objective function value: 115.68203631442458
0.785
Optimal values:
a = 0.2490389997702134
b = 0.3933307345782202
c = 0.3576302656515665
Objective function value: 108.45326582545438
0.786
Optimal values:
a = 0.17692872022246842
b = 0.5301500079002968
c = 0.29292127187726874
Objective function value: 120.37306978485996
0.791
Optimal values:
a = 0.0
b = 0.5464624971163495
c = 0.4535375028836504
Objective function value: 104.21885999090672
0.806
Optimal values:
a = 0.15251661817399595
b = 0.5993299520113485
c = 0.2481534298146555
Objective function value: 108.43176313399067
0.798


In [7]:
print(np.mean(results)*100, np.std(results)*100)

79.32000000000001 0.7884161337770818


In [3]:
import os
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from datasets import load_dataset, concatenate_datasets, DatasetDict
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics import f1_score


def objective_function(x, res, ground_truth):
    a, b, c = x
    return np.sum((a * res[0] + b * res[1] + c * res[2] - ground_truth) ** 2)

def constraint(x):
    return np.sum(x) - 1

results = []
spearman_results = []
pearson_results = []

results_baseline = []
results_crlft = []
results_sequential = []

spearmans_baseline = []
spearmans_crlft = []
spearmans_sequential = []

pearsons_baseline = []
pearsons_crlft = []
pearsons_sequential = []

alpha_beta_gamma = []


for n in range(5):

    np.random.seed(42)
    use_mnli = False
    result_path = 'paper_results_mnli/' if use_mnli else 'paper_results/'
    path = '/data1/malto/shroom'


    baseline_test = pd.read_csv(result_path + '/baseline_test' + str(n) + '.csv')['baseline'].to_numpy()
    baseline_val = pd.read_csv(result_path + '/baseline_val' + str(n) + '.csv')['baseline'].to_numpy()
    crlft_test = pd.read_csv(result_path + '/crlft_test' + str(n) + '.csv')['crlft'].to_numpy()
    crlft_val = pd.read_csv(result_path + '/crlft_val' + str(n) + '.csv')['crlft'].to_numpy()
    sequential_test = pd.read_csv(result_path + '/sequential_test' + str(n) + '.csv')['sequential'].to_numpy() 
    sequential_val = pd.read_csv(result_path + '/sequential_val' + str(n) + '.csv')['sequential'].to_numpy()

    labels_val = pd.read_json(path + '/val.model-aware.json', lines=True)['p(Hallucination)'].to_numpy()
    # read test.model-agnostic.json 

    #baseline_test = np.where(baseline_test > 0.5, 1, 0)
    #crlft_test = np.where(crlft_test > 0.5, 1, 0)
    #sequential_test = np.where(sequential_test > 0.5, 1, 0)
    res = np.stack((baseline_test, crlft_test, sequential_test), axis=0)
    test = load_dataset("json", data_files=["test.model-agnostic.json"])
    # shuffle the dataset
    ground_truth = test['train']['p(Hallucination)']
    ground_truth = np.array(ground_truth)
    #ground_truth = np.where(ground_truth > 0.5, 1, 0)

    # shuffle in the same way the ground truth and the predictions
    indices = np.arange(len(ground_truth))
    np.random.shuffle(indices)

    ground_truth = ground_truth[indices]
    res = res[:, indices]

    gt_ensemble = ground_truth[:500]
    res_ensemble = res[:, :500]

    gt_res = ground_truth[500:]
    res_res = res[:, 500:]

    con = {'type': 'eq', 'fun': constraint}
    bounds = [(0, 1), (0, 1), (0, 1)]  # a, b, c >= 0 and <= 1
    x0 = np.array([1/3, 1/3, 1/3])

    result = minimize(objective_function, x0, args=(res, ground_truth), bounds=bounds, constraints=con)

    # print("Optimal values:")
    # print("a =", result.x[0])
    # print("b =", result.x[1])
    # print("c =", result.x[2])
    # print("Objective function value:", result.fun)

    r1_new = result.x[0] * res_res[0]
    r2_new = result.x[1] * res_res[1]
    r3_new = result.x[2] * res_res[2]

    res = r1_new + r2_new + r3_new

    predictions = np.where(res > 0.5, 1, 0)
    labels = np.where(gt_res > 0.5, 1, 0)
    print(np.sum(predictions == labels)/len(labels))
    results.append(f1_score(labels, predictions))

    results_baseline.append(f1_score(np.where(res_res[0] > 0.5, 1, 0), labels))
    results_crlft.append(f1_score(np.where(res_res[1] > 0.5, 1, 0), labels))
    results_sequential.append(f1_score(np.where(res_res[2] > 0.5, 1, 0), labels))

    #spearman_results.append(spearmanr(res, gt_res)[0])
    #pearson_results.append(pearsonr(res, gt_res)[0])

    predictions = np.where(res_res[0] > 0.5, 1, 0)
    labels = np.where(gt_res > 0.5, 1, 0)
    # f1 score
    f1_score 
    
    alpha_beta_gamma.append(result.x)

0.785
0.786
0.791
0.806
0.798


In [5]:
print(np.mean(results_baseline)*100, np.std(results_baseline)*100)
print(np.mean(results_crlft)*100, np.std(results_crlft)*100)
print(np.mean(results_sequential)*100, np.std(results_sequential)*100)
print(np.mean(results)*100, np.std(results)*100)

63.41915974732161 4.089413816973061
62.906635927547825 3.614050579499803
71.88932197439028 1.9624354385852476
70.51915109916833 1.4994041646829679


In [1]:
import os
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from datasets import load_dataset, concatenate_datasets, DatasetDict
from scipy.stats import spearmanr, pearsonr


def objective_function(x, res, ground_truth):
    a, b, c = x
    return np.sum((a * res[0] + b * res[1] + c * res[2] - ground_truth) ** 2)

def constraint(x):
    return np.sum(x) - 1

results = []
spearman_results = []
pearson_results = []

results_baseline = []
results_crlft = []
results_sequential = []

spearmans_baseline = []
spearmans_crlft = []
spearmans_sequential = []

pearsons_baseline = []
pearsons_crlft = []
pearsons_sequential = []

alpha_beta_gamma = []

all_results = []

for n in range(5):
    use_mnli = True
    result_path = 'paper_results_mnli/' if use_mnli else 'paper_results/'
    path = '/data1/malto/shroom'


    baseline_test = pd.read_csv(result_path + '/baseline_test' + str(n) + '.csv')['baseline'].to_numpy()
    crlft_test = pd.read_csv(result_path + '/crlft_test' + str(n) + '.csv')['crlft'].to_numpy()
    sequential_test = pd.read_csv(result_path + '/sequential_test' + str(n) + '.csv')['sequential'].to_numpy() 
    labels_val = pd.read_json(path + '/val.model-aware.json', lines=True)['p(Hallucination)'].to_numpy()
    test = load_dataset("json", data_files=["test.model-agnostic.json"])
    # shuffle the dataset
    ground_truth = test['train']['p(Hallucination)']
    ground_truth = np.array(ground_truth)
    res = np.stack((baseline_test, crlft_test, sequential_test, ground_truth), axis=0)
    
    res = np.expand_dims(res, axis=0)

    if n == 0: 
        all_results = res
    else:
        all_results = np.concatenate((all_results, res), axis=0)

# save pickle file
import pickle
with open('all_results_mnli.pkl', 'wb') as f:
    pickle.dump(all_results, f)

/data1/malto/csavelli/venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
with open('all_results.pkl', 'rb') as f:
    all_results2 = pickle.load(f)